In [1]:
from __future__ import print_function
#%matplotlib inline
import argparse
import os
import random
import math
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from torchvision.utils import save_image


In [2]:
# Root directory for dataset
dataroot = "data"

# Number of workers for dataloader
workers = 2

# Batch size during training
batch_size = 50

# Spatial size of training images. All images will be resized to this
#   size using a transformer.
image_size = 32

# Number of channels in the training images. For color images this is 3
nc = 1

# Size of z latent vector (i.e. size of generator input)
nz = 100

# Size of feature maps in generator
ngf = 64

# Size of feature maps in discriminator
ndf = 64

# Number of training epochs
num_epochs = 5

# Learning rate for optimizers
lr = 0.0002

# Beta1 hyperparam for Adam optimizers
beta1 = 0.5

# Number of GPUs available. Use 0 for CPU mode.
ngpu = 1

# Number of classes in the training set.
n_classes = 10

# The interval between image sampling.
sample_interval = 400 

In [3]:
os.makedirs("images", exist_ok=True)
os.makedirs(dataroot, exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    dset.MNIST(
        dataroot,
        train=True,
        download=True,
        transform=transforms.Compose(
            [transforms.Resize(image_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
        ),
    ),
    batch_size=batch_size,
    shuffle=True,
)

In [4]:

# Create the dataloader
dataloaderC = torch.utils.data.DataLoader(dataloader, batch_size=batch_size,
                                         shuffle=True, num_workers=workers)

# Decide which device we want to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")


In [5]:
# custom weights initialization called on netG and netD
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [6]:
image_shape = (nc, image_size, image_size)
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, n_classes)

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(nz + n_classes, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(image_shape))),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        # Concatenate label embedding and image to produce input
        gen_input = torch.cat((self.label_emb(labels), noise), -1)
        img = self.model(gen_input)
        img = img.view(img.size(0), *image_shape)
        return img


In [7]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.label_embedding = nn.Embedding(n_classes, n_classes)

        self.model = nn.Sequential(
            nn.Linear(n_classes + int(np.prod(image_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.Dropout(0.4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 1),
        )

    def forward(self, img, labels):
        # Concatenate label embedding and image to produce input
        d_in = torch.cat((img.view(img.size(0), -1), self.label_embedding(labels)), -1)
        validity = self.model(d_in)
        return validity


In [8]:
# Create the generators
netG = Generator().to(device)

# Initialize weights
netG.apply(weights_init)

# Print the model
print(netG)

Generator(
  (label_emb): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=110, out_features=128, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Linear(in_features=128, out_features=256, bias=True)
    (3): BatchNorm1d(256, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Linear(in_features=256, out_features=512, bias=True)
    (6): BatchNorm1d(512, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Linear(in_features=512, out_features=1024, bias=True)
    (9): BatchNorm1d(1024, eps=0.8, momentum=0.1, affine=True, track_running_stats=True)
    (10): LeakyReLU(negative_slope=0.2, inplace)
    (11): Linear(in_features=1024, out_features=1024, bias=True)
    (12): Tanh()
  )
)


In [9]:
# Create the discriminators
netD = Discriminator().to(device)

# Initialize weights
netD.apply(weights_init)

# Print the model
print(netD)

Discriminator(
  (label_embedding): Embedding(10, 10)
  (model): Sequential(
    (0): Linear(in_features=1034, out_features=512, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Dropout(p=0.4)
    (4): LeakyReLU(negative_slope=0.2, inplace)
    (5): Linear(in_features=512, out_features=512, bias=True)
    (6): Dropout(p=0.4)
    (7): LeakyReLU(negative_slope=0.2, inplace)
    (8): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [10]:
# Initialize MSELoss function
loss = nn.MSELoss()

# Establish convention for real and fake labels during training
real_label = 1
fake_label = 0

# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(beta1, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(beta1, 0.999))

In [11]:
FloatTensor = torch.cuda.FloatTensor 
LongTensor = torch.cuda.LongTensor


def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row ** 2, nz))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = netG(z, labels)
    save_image(gen_imgs.data, "images/%d.png" % batches_done, nrow=n_row, normalize=True)


In [12]:

# ----------
#  Training
# ----------

for epoch in range(num_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizerG.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, nz))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = netG(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity = netD(gen_imgs, gen_labels)
        g_loss = loss(validity, valid)

        g_loss.backward()
        optimizerG.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizerD.zero_grad()

        # Loss for real images
        validity_real = netD(real_imgs, labels)
        d_real_loss = loss(validity_real, valid)

        # Loss for fake images
        validity_fake = netD(gen_imgs.detach(), gen_labels)
        d_fake_loss = loss(validity_fake, fake)

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        d_loss.backward()
        optimizerD.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, num_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

[Epoch 0/5] [Batch 0/1200] [D loss: 0.414420] [G loss: 0.907471]
[Epoch 0/5] [Batch 1/1200] [D loss: 0.224121] [G loss: 0.888766]
[Epoch 0/5] [Batch 2/1200] [D loss: 0.079183] [G loss: 0.872494]
[Epoch 0/5] [Batch 3/1200] [D loss: 0.011845] [G loss: 0.849592]
[Epoch 0/5] [Batch 4/1200] [D loss: 0.026422] [G loss: 0.840078]
[Epoch 0/5] [Batch 5/1200] [D loss: 0.026559] [G loss: 0.831851]
[Epoch 0/5] [Batch 6/1200] [D loss: 0.013502] [G loss: 0.842605]
[Epoch 0/5] [Batch 7/1200] [D loss: 0.014412] [G loss: 0.828591]
[Epoch 0/5] [Batch 8/1200] [D loss: 0.011509] [G loss: 0.831327]
[Epoch 0/5] [Batch 9/1200] [D loss: 0.011811] [G loss: 0.813075]
[Epoch 0/5] [Batch 10/1200] [D loss: 0.014303] [G loss: 0.812234]
[Epoch 0/5] [Batch 11/1200] [D loss: 0.012958] [G loss: 0.798762]
[Epoch 0/5] [Batch 12/1200] [D loss: 0.014668] [G loss: 0.800444]
[Epoch 0/5] [Batch 13/1200] [D loss: 0.016748] [G loss: 0.786497]
[Epoch 0/5] [Batch 14/1200] [D loss: 0.016825] [G loss: 0.772719]
[Epoch 0/5] [Batch 1

[Epoch 0/5] [Batch 125/1200] [D loss: 0.191529] [G loss: 0.474538]
[Epoch 0/5] [Batch 126/1200] [D loss: 0.177322] [G loss: 0.316476]
[Epoch 0/5] [Batch 127/1200] [D loss: 0.156213] [G loss: 0.292673]
[Epoch 0/5] [Batch 128/1200] [D loss: 0.118957] [G loss: 0.459053]
[Epoch 0/5] [Batch 129/1200] [D loss: 0.115576] [G loss: 0.549726]
[Epoch 0/5] [Batch 130/1200] [D loss: 0.079530] [G loss: 0.623585]
[Epoch 0/5] [Batch 131/1200] [D loss: 0.079147] [G loss: 0.542009]
[Epoch 0/5] [Batch 132/1200] [D loss: 0.075195] [G loss: 0.550735]
[Epoch 0/5] [Batch 133/1200] [D loss: 0.090176] [G loss: 0.710728]
[Epoch 0/5] [Batch 134/1200] [D loss: 0.104591] [G loss: 0.439457]
[Epoch 0/5] [Batch 135/1200] [D loss: 0.108056] [G loss: 0.693486]
[Epoch 0/5] [Batch 136/1200] [D loss: 0.172523] [G loss: 0.232391]
[Epoch 0/5] [Batch 137/1200] [D loss: 0.323772] [G loss: 1.093933]
[Epoch 0/5] [Batch 138/1200] [D loss: 0.583365] [G loss: 0.011794]
[Epoch 0/5] [Batch 139/1200] [D loss: 0.167054] [G loss: 0.527

[Epoch 0/5] [Batch 249/1200] [D loss: 0.143533] [G loss: 0.684107]
[Epoch 0/5] [Batch 250/1200] [D loss: 0.204222] [G loss: 0.173830]
[Epoch 0/5] [Batch 251/1200] [D loss: 0.222947] [G loss: 0.903129]
[Epoch 0/5] [Batch 252/1200] [D loss: 0.164363] [G loss: 0.217744]
[Epoch 0/5] [Batch 253/1200] [D loss: 0.092127] [G loss: 0.564853]
[Epoch 0/5] [Batch 254/1200] [D loss: 0.084602] [G loss: 0.675145]
[Epoch 0/5] [Batch 255/1200] [D loss: 0.109858] [G loss: 0.417949]
[Epoch 0/5] [Batch 256/1200] [D loss: 0.073021] [G loss: 0.649465]
[Epoch 0/5] [Batch 257/1200] [D loss: 0.083366] [G loss: 0.589252]
[Epoch 0/5] [Batch 258/1200] [D loss: 0.083142] [G loss: 0.534873]
[Epoch 0/5] [Batch 259/1200] [D loss: 0.136524] [G loss: 0.565233]
[Epoch 0/5] [Batch 260/1200] [D loss: 0.205010] [G loss: 0.230586]
[Epoch 0/5] [Batch 261/1200] [D loss: 0.384644] [G loss: 1.435378]
[Epoch 0/5] [Batch 262/1200] [D loss: 0.378233] [G loss: 0.036234]
[Epoch 0/5] [Batch 263/1200] [D loss: 0.147290] [G loss: 0.451

[Epoch 0/5] [Batch 372/1200] [D loss: 0.121874] [G loss: 0.500581]
[Epoch 0/5] [Batch 373/1200] [D loss: 0.113327] [G loss: 0.510617]
[Epoch 0/5] [Batch 374/1200] [D loss: 0.106737] [G loss: 0.484804]
[Epoch 0/5] [Batch 375/1200] [D loss: 0.142765] [G loss: 0.573502]
[Epoch 0/5] [Batch 376/1200] [D loss: 0.128718] [G loss: 0.378267]
[Epoch 0/5] [Batch 377/1200] [D loss: 0.129365] [G loss: 0.628542]
[Epoch 0/5] [Batch 378/1200] [D loss: 0.151455] [G loss: 0.316360]
[Epoch 0/5] [Batch 379/1200] [D loss: 0.144486] [G loss: 0.588330]
[Epoch 0/5] [Batch 380/1200] [D loss: 0.138131] [G loss: 0.384898]
[Epoch 0/5] [Batch 381/1200] [D loss: 0.106510] [G loss: 0.552988]
[Epoch 0/5] [Batch 382/1200] [D loss: 0.127128] [G loss: 0.540746]
[Epoch 0/5] [Batch 383/1200] [D loss: 0.125596] [G loss: 0.394101]
[Epoch 0/5] [Batch 384/1200] [D loss: 0.144339] [G loss: 0.725894]
[Epoch 0/5] [Batch 385/1200] [D loss: 0.210385] [G loss: 0.216184]
[Epoch 0/5] [Batch 386/1200] [D loss: 0.254821] [G loss: 1.141

[Epoch 0/5] [Batch 496/1200] [D loss: 0.093304] [G loss: 0.620534]
[Epoch 0/5] [Batch 497/1200] [D loss: 0.100410] [G loss: 0.537974]
[Epoch 0/5] [Batch 498/1200] [D loss: 0.118520] [G loss: 0.790169]
[Epoch 0/5] [Batch 499/1200] [D loss: 0.139562] [G loss: 0.306869]
[Epoch 0/5] [Batch 500/1200] [D loss: 0.129169] [G loss: 0.707101]
[Epoch 0/5] [Batch 501/1200] [D loss: 0.115859] [G loss: 0.435162]
[Epoch 0/5] [Batch 502/1200] [D loss: 0.104165] [G loss: 0.700749]
[Epoch 0/5] [Batch 503/1200] [D loss: 0.102473] [G loss: 0.585751]
[Epoch 0/5] [Batch 504/1200] [D loss: 0.113363] [G loss: 0.559990]
[Epoch 0/5] [Batch 505/1200] [D loss: 0.116562] [G loss: 0.561847]
[Epoch 0/5] [Batch 506/1200] [D loss: 0.142076] [G loss: 0.352095]
[Epoch 0/5] [Batch 507/1200] [D loss: 0.163862] [G loss: 1.013199]
[Epoch 0/5] [Batch 508/1200] [D loss: 0.374094] [G loss: 0.053378]
[Epoch 0/5] [Batch 509/1200] [D loss: 0.366092] [G loss: 1.436067]
[Epoch 0/5] [Batch 510/1200] [D loss: 0.145425] [G loss: 0.343

...

[Epoch 4/5] [Batch 656/1200] [D loss: 0.193300] [G loss: 0.306734]
[Epoch 4/5] [Batch 657/1200] [D loss: 0.180505] [G loss: 0.433749]
[Epoch 4/5] [Batch 658/1200] [D loss: 0.174122] [G loss: 0.493272]
[Epoch 4/5] [Batch 659/1200] [D loss: 0.206827] [G loss: 0.289094]
[Epoch 4/5] [Batch 660/1200] [D loss: 0.208234] [G loss: 0.480944]
[Epoch 4/5] [Batch 661/1200] [D loss: 0.138581] [G loss: 0.407747]
[Epoch 4/5] [Batch 662/1200] [D loss: 0.169520] [G loss: 0.495054]
[Epoch 4/5] [Batch 663/1200] [D loss: 0.178995] [G loss: 0.447125]
[Epoch 4/5] [Batch 664/1200] [D loss: 0.165876] [G loss: 0.408589]
[Epoch 4/5] [Batch 665/1200] [D loss: 0.178473] [G loss: 0.498829]
[Epoch 4/5] [Batch 666/1200] [D loss: 0.174116] [G loss: 0.337063]
[Epoch 4/5] [Batch 667/1200] [D loss: 0.162598] [G loss: 0.520329]
[Epoch 4/5] [Batch 668/1200] [D loss: 0.168863] [G loss: 0.561278]
[Epoch 4/5] [Batch 669/1200] [D loss: 0.187720] [G loss: 0.391257]
[Epoch 4/5] [Batch 670/1200] [D loss: 0.167863] [G loss: 0.434

[Epoch 4/5] [Batch 782/1200] [D loss: 0.169263] [G loss: 0.609941]
[Epoch 4/5] [Batch 783/1200] [D loss: 0.188078] [G loss: 0.518107]
[Epoch 4/5] [Batch 784/1200] [D loss: 0.168566] [G loss: 0.335232]
[Epoch 4/5] [Batch 785/1200] [D loss: 0.201857] [G loss: 0.614996]
[Epoch 4/5] [Batch 786/1200] [D loss: 0.168494] [G loss: 0.451425]
[Epoch 4/5] [Batch 787/1200] [D loss: 0.165847] [G loss: 0.388044]
[Epoch 4/5] [Batch 788/1200] [D loss: 0.149540] [G loss: 0.494515]
[Epoch 4/5] [Batch 789/1200] [D loss: 0.170637] [G loss: 0.503979]
[Epoch 4/5] [Batch 790/1200] [D loss: 0.187595] [G loss: 0.445591]
[Epoch 4/5] [Batch 791/1200] [D loss: 0.164725] [G loss: 0.508426]
[Epoch 4/5] [Batch 792/1200] [D loss: 0.166852] [G loss: 0.509632]
[Epoch 4/5] [Batch 793/1200] [D loss: 0.172338] [G loss: 0.463561]
[Epoch 4/5] [Batch 794/1200] [D loss: 0.190591] [G loss: 0.350105]
[Epoch 4/5] [Batch 795/1200] [D loss: 0.212203] [G loss: 0.622110]
[Epoch 4/5] [Batch 796/1200] [D loss: 0.172278] [G loss: 0.343

[Epoch 4/5] [Batch 905/1200] [D loss: 0.169157] [G loss: 0.505942]
[Epoch 4/5] [Batch 906/1200] [D loss: 0.157962] [G loss: 0.399372]
[Epoch 4/5] [Batch 907/1200] [D loss: 0.170462] [G loss: 0.525345]
[Epoch 4/5] [Batch 908/1200] [D loss: 0.168682] [G loss: 0.472455]
[Epoch 4/5] [Batch 909/1200] [D loss: 0.215725] [G loss: 0.300518]
[Epoch 4/5] [Batch 910/1200] [D loss: 0.232799] [G loss: 0.940271]
[Epoch 4/5] [Batch 911/1200] [D loss: 0.318447] [G loss: 0.106838]
[Epoch 4/5] [Batch 912/1200] [D loss: 0.204237] [G loss: 0.762019]
[Epoch 4/5] [Batch 913/1200] [D loss: 0.182726] [G loss: 0.539321]
[Epoch 4/5] [Batch 914/1200] [D loss: 0.169831] [G loss: 0.336645]
[Epoch 4/5] [Batch 915/1200] [D loss: 0.161600] [G loss: 0.366844]
[Epoch 4/5] [Batch 916/1200] [D loss: 0.181038] [G loss: 0.517229]
[Epoch 4/5] [Batch 917/1200] [D loss: 0.152239] [G loss: 0.417118]
[Epoch 4/5] [Batch 918/1200] [D loss: 0.155102] [G loss: 0.548922]
[Epoch 4/5] [Batch 919/1200] [D loss: 0.182333] [G loss: 0.462

[Epoch 4/5] [Batch 1029/1200] [D loss: 0.170201] [G loss: 0.495137]
[Epoch 4/5] [Batch 1030/1200] [D loss: 0.180601] [G loss: 0.292348]
[Epoch 4/5] [Batch 1031/1200] [D loss: 0.167566] [G loss: 0.569854]
[Epoch 4/5] [Batch 1032/1200] [D loss: 0.146057] [G loss: 0.411148]
[Epoch 4/5] [Batch 1033/1200] [D loss: 0.165412] [G loss: 0.487811]
[Epoch 4/5] [Batch 1034/1200] [D loss: 0.157567] [G loss: 0.364448]
[Epoch 4/5] [Batch 1035/1200] [D loss: 0.190087] [G loss: 0.567997]
[Epoch 4/5] [Batch 1036/1200] [D loss: 0.170236] [G loss: 0.384335]
[Epoch 4/5] [Batch 1037/1200] [D loss: 0.181954] [G loss: 0.480734]
[Epoch 4/5] [Batch 1038/1200] [D loss: 0.160025] [G loss: 0.633158]
[Epoch 4/5] [Batch 1039/1200] [D loss: 0.172075] [G loss: 0.325564]
[Epoch 4/5] [Batch 1040/1200] [D loss: 0.173293] [G loss: 0.682106]
[Epoch 4/5] [Batch 1041/1200] [D loss: 0.170732] [G loss: 0.361889]
[Epoch 4/5] [Batch 1042/1200] [D loss: 0.161303] [G loss: 0.461993]
[Epoch 4/5] [Batch 1043/1200] [D loss: 0.139168]

[Epoch 4/5] [Batch 1153/1200] [D loss: 0.177252] [G loss: 0.441133]
[Epoch 4/5] [Batch 1154/1200] [D loss: 0.208013] [G loss: 0.536289]
[Epoch 4/5] [Batch 1155/1200] [D loss: 0.196698] [G loss: 0.276302]
[Epoch 4/5] [Batch 1156/1200] [D loss: 0.176780] [G loss: 0.492117]
[Epoch 4/5] [Batch 1157/1200] [D loss: 0.185354] [G loss: 0.537649]
[Epoch 4/5] [Batch 1158/1200] [D loss: 0.183561] [G loss: 0.306171]
[Epoch 4/5] [Batch 1159/1200] [D loss: 0.167243] [G loss: 0.623145]
[Epoch 4/5] [Batch 1160/1200] [D loss: 0.181528] [G loss: 0.530632]
[Epoch 4/5] [Batch 1161/1200] [D loss: 0.178211] [G loss: 0.354540]
[Epoch 4/5] [Batch 1162/1200] [D loss: 0.154071] [G loss: 0.352618]
[Epoch 4/5] [Batch 1163/1200] [D loss: 0.165498] [G loss: 0.568549]
[Epoch 4/5] [Batch 1164/1200] [D loss: 0.160096] [G loss: 0.461687]
[Epoch 4/5] [Batch 1165/1200] [D loss: 0.166742] [G loss: 0.447149]
[Epoch 4/5] [Batch 1166/1200] [D loss: 0.154060] [G loss: 0.508104]
[Epoch 4/5] [Batch 1167/1200] [D loss: 0.166669]